In [11]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from rich.console import Console
from rich.table import Table
from rich.markdown import Markdown

In [20]:
# # Define the prompts
prompts = [
    "A farmer has chickens and cows on his farm. There are a total of 30 heads and 100 legs. How many chickens and how many cows are there?",
    "Observe the following sequence of numbers: 2, 4, 8, 16. Predict the next number in the sequence and explain the pattern.",
    "Hand is to glove as foot is to ____?"
]

In [21]:
# Load the TinyLlama model and tokenizer
tinyllama_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tinyllama_model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# Load the DeepSeek R1 Distill Qwen 1.5B model and tokenizer
deepseek_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
deepseek_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

In [ ]:
from rich.console import Console
from rich.markdown import Markdown

# Function to generate a response from a model
def generate_response(model, tokenizer, prompt):
    # Enhance the prompt with summarization instructions
    instruction = "Please provide a complete and concise summary of the following within 512 tokens:\n"
    enhanced_prompt = instruction + prompt

    # Tokenize the enhanced prompt
    inputs = tokenizer(enhanced_prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,  # Limit the generated response to 512 tokens
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )
    # Extract the generated tokens, excluding the input prompt
    generated_tokens = outputs[:, inputs['input_ids'].shape[-1]:]
    response = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    return response

# Create a list to store the results
results = []

# Iterate over the prompts and get responses from both models
for prompt in prompts:
    tinyllama_response = generate_response(tinyllama_model, tinyllama_tokenizer, prompt)
    deepseek_response = generate_response(deepseek_model, deepseek_tokenizer, prompt)
    results.append({
        "Prompt": prompt,
        "TinyLlama": tinyllama_response,
        "DeepSeek R1 Distill Qwen 1.5B": deepseek_response
    })

# Initialize the console for rich output
console = Console()

# Display each prompt and its corresponding responses
for result in results:
    markdown_content = f"""
**Prompt:**

{result['Prompt']}

**TinyLlama:**

{result['TinyLlama']}

**DeepSeek R1 Distill Qwen 1.5B:**

{result['DeepSeek R1 Distill Qwen 1.5B']}
"""
    console.print(Markdown(markdown_content))
